In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import random
import geatpy as ea
import matplotlib.pyplot as plt
import time
import xlwt

In [2]:
class flexibleSchedulingASC:
    def __init__(self):
     
        self.mDir = '../data/'
        self.mDir_input = 'input/'
        self.mDir_solution = 'solution/'
        self.mData = {}

    def dataGeneration(self, N, NK, NA, NB, NAME, AGV, ASC):
        dataset = pd.read_excel(self.mDir + self.mDir_input + 'AGV-ASC flexiable interaction.xlsx',
                                sheet_name=NAME, skiprows=0)
        df = pd.DataFrame(dataset)
        dataExp = df.loc[1: 7, ['No', 'type', 'XO', 'YO', 'XD', 'YD']]

        J = list(df['No'].values)[1:N + 1]
        TP = list(df['type'].values)[0:N + 1]
        XO = list(df['XO'].values)[0:N + 1]
        YO = list(df['YO'].values)[0:N + 1]
        XD = list(df['XD'].values)[0:N + 1]
        YD = list(df['YD'].values)[0:N + 1]

        JS = list(range(0, N + 1))
        JE = list(range(1, N + 2))
        JD = list(range(0, N + 2))

        K = set(list(range(1, NK + 1)))
        B = set(list(range(1, NB + 1)))
        A = set(list(range(1, NA + 1)))

        JI = [i for i, x in enumerate(TP) if x == 1]  # 进口箱
        JO = [j for j, x in enumerate(TP) if x == 0]  # 出口箱

        Container = {'Len': 12, 'Wdh': 3}  # 集装箱规格：长、宽

        LocB = (200, 215)  # 缓冲区B的位置坐标(x,y)
        BayB = (0, 5)  # 缓冲区B在箱区中的位置(贝位，列位)
        TTKN = np.zeros([N + 1, 1])
        TTYN = np.zeros([N + 1, 1])
        TTK, TTY = [], []
        for i in J:
            if i in JI:
                TTKN[i][0] = (1 / AGV['SPD']) * (abs(XO[i] - LocB[0]) + abs(YO[i] - LocB[1]))
            elif i in JO:
                TTKN[i][0] = (1 / AGV['SPD']) * (abs(XD[i] - LocB[0]) + abs(YD[i] - LocB[1]))

        for i in J:
            if i in JI:
                TTYN[i][0] = (1 / ASC['SPD']) * (
                        (XD[i] - BayB[0]) * Container['Len'] + abs(YD[i] - BayB[1]) * Container['Wdh'])
            elif i in JO:
                TTYN[i][0] = (1 / ASC['SPD']) * (
                        (XO[i] - BayB[0]) * Container['Len'] + abs(YO[i] - BayB[1]) * Container['Wdh'])

        for j in range(N + 1):
            TTK.append(float(TTKN[j][0]))
            TTY.append(float(TTYN[j][0]))

        TK, TY = np.zeros([N + 2, N + 2]), np.zeros([N + 2, N + 2])

        for a in range(N + 2):
            for b in range(N + 2):
                if a in JI:
                    if b in JI:
                        TK[a][b] = (1 / AGV['SPD']) * (abs(LocB[0] - XO[b]) + abs(LocB[1] - YO[b]))
                        TY[a][b] = (1 / ASC['SPD']) * (
                                abs(XD[a] - BayB[0]) * Container['Len'] + abs(YD[a] - BayB[1]) * Container['Wdh'])
                    elif b in JO:
                        # TK[a][b] = (1/AGV['SPD']) * (abs(LocB[0] - XD[b]) + abs(LocB[1] - YD[b]))
                        TK[a][b] = 0
                        TY[a][b] = (1 / ASC['SPD']) * (
                                abs(XD[a] - XO[b]) * Container['Len'] + abs(YD[a] - YO[b]) * Container['Wdh'])
                elif a in JO:
                    if b in JI:
                        TK[a][b] = (1 / AGV['SPD']) * (abs(XD[a] - XO[b]) + abs(YD[a] - YO[b]))
                        TY[a][b] = (1 / ASC['SPD']) * (
                                abs(BayB[0] - XD[b]) * Container['Len'] + abs(BayB[1] - YD[b]) * Container['Wdh'])
                    elif b in JO:
                        TK[a][b] = (1 / AGV['SPD']) * (abs(XD[a] - LocB[0]) + abs(YD[a] - LocB[1]))
                        TY[a][b] = (1 / ASC['SPD']) * (
                                abs(BayB[0] - XO[b]) * Container['Len'] + abs(BayB[1] - YO[b]) * Container['Wdh'])

        for a in range(N + 2):
            for b in range(N + 2):
                if a == 0:
                    TK[a][b] = 0
                    TY[a][b] = 0
                if b == 0:
                    TK[a][b] = 9999
                    TY[a][b] = 9999
                if b == N + 1:
                    TK[a][b] = 0
                    TY[a][b] = 0
                if a == N + 1:
                    TK[a][b] = 9999
                    TY[a][b] = 9999
        TK[0][N + 1] = 9999
        TY[0][N + 1] = 9999

        self.mData['J'], self.mData['JI'], self.mData['JO'] = J, JI, JO
        self.mData['K'], self.mData['B'], self.mData['A'] = K, B, A
        self.mData['TTK'], self.mData['TTY'] = TTK, TTY
        self.mData['TK'], self.mData['TY'] = TK, TY
        self.lenQ = N
        self.mData['AGV'], self.mData['ASC'] = AGV, ASC
        # return J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, dataExp

    def getParam(self):
        return (self.mData["J"], self.mData["JI"], self.mData["JO"], self.mData["K"],
                self.mData["B"], self.mData["A"], self.mData["TTK"], self.mData["TTY"],
                self.mData["TK"], self.mData["TY"], self.mData['AGV'], self.mData['ASC'])

    def assignMent(self, sequence):
        (J, JI, JO, K, B, A, TTK, TTY, TK, TY, AGV, ASC) = self.getParam()
        makespan, cpu = 9999, 0

        # JI = [2, 3, 5, 6, 8]      进口箱
        # JO = [1, 4, 7, 9, 10]     出口箱
        # random.seed(sed)
        # sequence = random.sample(range(1, N + 1), N)   # 示例序列 (假定为ASC作业序列)

        V = {k: [] for k in K}      # AGV作业任务序列, e.g., V = {1:[2,3,4], 2:[5,7,9], ...}
        Q = []                      # ASC 作业序列
        SA = {a: [] for a in A}     # 硬交互位置任务进出序列, e.g., SA = {1:[2,3,4], ...}
        SB = {b: [] for b in B}     # 软交互位置任务进出序列, e.g., SB = {1:[5,7,9], 2:[6, 10]...}
        TVS, TVE = {k: [] for k in K}, {k: [] for k in K}     # AGV 作业任务的开始时间和结束时间
        TAS, TAE = {a: [] for a in A}, {a: [] for a in A}     # 在硬交互位中, 任务进入和离开的时间
        TBS, TBE = {b: [] for b in B}, {b: [] for b in B}     # 在软交互位中, 任务进入和离开的时间
        TYS, TYE = [], []                                   # ASC 作业任务的开始时间和结束时间

        ######## 初始化任务序列与作业时间
        for k in K:
            V[k].append(0)
            TVS[k].append(0)
            TVE[k].append(0)
        for a in A:
            TAS[a].append(0)
            TAE[a].append(0)
        for b in B:
            TBS[b].append(0)
            TBE[b].append(0)
        TYS.append(0)
        TYE.append(0)
        Q.append(0)

        KSS = {k: False for k in K}       # 车辆的状态. False:可用; Ture: 不可用(占用)
        ASS = {a: False for a in A}      # 硬交互位的状态. False:可用; Ture: 不可用(占用)
        BSS = {b: False for b in B}      # 软交互位的状态. False:可用; Ture: 不可用(占用)
        SSS = {1: False}                 # ASC的状态

        sequence=list(sequence)
        hangingSet = []
        while len(sequence) != 0:
            KB, AB, BB = 0, 0, 0 # AGV、硬交互位、软交互位的可用状态定位
            it = sequence.pop(0)
            if it in JI:    ###*** 进口箱任务JI, 起始在AGV  ***###
                ########****** 任务分配到任务量最小且空闲的车辆 *****#######
                if self.stateDecide(KSS) == True:
                    VX = []
                    EX = list(KSS.values())
                    [VX.append(i + 1) for i in range(len(EX)) if EX[i] == False]    # 查找空闲的 AGV
                    Vcount = [len(V[v]) for v in VX]
                    k = VX[Vcount.index(min(Vcount))]               # 任务分配给当前任务量最小的空闲的AGV
                    ### k = list(KSS.values()).index(False) + 1     # 任务随机分配给          空闲的AGV
                    V[k].append(it)     # 把任务分配给该空闲车辆
                    KSS[k] = True       # 更改车辆为占用状态
                    TVS[k].append(TVE[k][-1] + TK[V[k][-2]][it]) ### 起始时刻 =上一个任务的结束时间 +相邻两任务的travel time

                    ########****** 任务分配到交接区: A区 或 B区  *****######
                    if self.stateDecide(ASS) == True: # A区 直接交互
                        TVE[k].append(TVS[k][-1] + TTK[it] + ASC['BY'] + ASC['TY'])     ## AGV作业结束时间 (箱子直接在AGV上由ASC提走)
                        KSS[k] = False                              ## AGV释放到可用状态
                        a = list(ASS.values()).index(False) + 1     ## 任务分配到--直接交互位a
                        ASS[a] = True
                        SA[a].append(it)
                        TAS[a].append(TVE[k][-1] - ASC['BY'] - ASC['TY'])   # 进入直接交互位时间 = AGV作业完成时间 - ASC起吊时间 - ASC抓箱时间


                        ####### ******  任务由ASC作业  ****** ########
                        if self.stateDecide(SSS) == True: # ASC是否可用
                            Q.append(it)
                            ASS[a] = False
                            # ASC开始作业任务it的时间 = max{任务进入直接交互位的时间, 上一个任务的结束时间 +相邻两任务的travel time}
                            TYS.append(max(TAS[a][-1], TYE[-1] + TY[Q[-2]][it]))
                            TAE[a].append(TYS[-1])                          # 离开直接交互位时间 = ASC提箱完成的时刻
                            SSS[1] = True
                            TYE.append(TYS[-1] + 2 * ASC['BY'] + 3 * ASC['TY'] + TTY[it])
                            SSS[1] = False

                    elif self.stateDecide(BSS) == True: # B区 间接交互
                        TVE[k].append(TVS[k][-1] + TTK[it] + AGV['BK'])
                        b = list(BSS.values()).index(False) + 1             # 任务分配到--间接交互位b
                        BSS[b] = True
                        SB[b].append(it)
                        TBS[b].append(TVE[k][-1])                           # 进入缓冲区的时间 = AGV释放集装箱到支架上的结束时刻


                        ####### ******  任务由ASC作业  ****** ########
                        if self.stateDecide(SSS) == True:  # ASC是否可用
                            Q.append(it)
                            BSS[b] = False
                            ## ASC开始作业任务it的时间 = max{任务进入间接交互位的时间, 上一个任务的结束时间 +相邻两任务的travel time}
                            TYS.append(max(TBS[b][-1], TYE[-1] + TY[Q[-2]][it]))
                            TBE[b].append(TYS[-1])                          # 离开的缓冲区的时间 = ASC提箱完成的时刻
                            SSS[1] = True
                            TYE.append(TYS[-1] + 2 * ASC['BY'] + 3 * ASC['TY'] + TTY[it])
                            SSS[1] = False

                else:   # 若当前无可用车辆, 则悬挂任务i, 等待有可用的车辆
                    hangingSet.append(it)
                    print('some of task have no assig...')
                    sequence += hangingSet # 悬挂任务返回调度池 sequence


            elif it in JO:  ###*** 出口箱任务 JO, 起始在ASC  ***###
                if self.stateDecide(KSS) == True:  # 判断ASC是否空闲
                    Q.append(it)            # 把任务分配给ASC
                    SSS[1] = True           # 更改ASC状态
                    TYS.append(TYE[-1] + TY[Q[-2]][it])

                    ########****** 任务分配到交接区: A区 或 B区  *****######
                    if self.stateDecide(ASS) == True:   # A区
                        TYE.append(TYS[-1] + 2 * ASC['BY'] + 3 * ASC['TY'] + TTY[it])
                        SSS[1] = False
                        a = list(ASS.values()).index(False) + 1     ## 任务分配到--直接交互位a
                        ASS[a] = True
                        SA[a].append(it)
                        TAS[a].append(TYE[-1] - ASC['BY'] - ASC['TY'])   # 进入直接交互位时间 = AGV作业完成时间 - ASC落吊时间 - ASC放箱时间

                        ########****** 任务分配到任务量最小且空闲的车辆 *****#######
                        if self.stateDecide(KSS) == True:
                            VX = []
                            EX = list(KSS.values())
                            [VX.append(i + 1) for i in range(len(EX)) if EX[i] == False]  # 查找空闲的 AGV
                            Vcount = [len(V[v]) for v in VX]
                            k = VX[Vcount.index(min(Vcount))]               # 任务分配给当前任务量最小的空闲的AGV
                            ### k = list(KSS.values()).index(False) + 1     # 任务随机分配给          空闲的AGV
                            V[k].append(it)     # 把任务分配给该空闲车辆
                            KSS[k] = True       # 更改车辆为占用状态
                            ## AGV开始作业任务it的时间 = max{任务进入直接交互位的时间, 上一个任务的结束时间 + 相邻两任务的travel time}
                            TVS[k].append(max(TAS[a][-1], TVE[k][-1] + TK[V[k][-2]][it]))
                            TAE[a].append(TVS[k][-1])       # 任务离开直接交互位的时间
                            ASS[a] = False
                            TVE[k].append(TVS[k][-1] + TTK[it] + AGV['TQ'])
                            KSS[k] = False      # 复位车辆状态

                    elif self.stateDecide(BSS) == True: # B区
                        TYE.append(TYS[-1] + TTY[it] + 2 * ASC['BY'] + 3 * ASC['TY'])
                        SSS[1] = False
                        b = list(BSS.values()).index(False) + 1     # 任务分配到--间接交互位b
                        BSS[b] = True
                        SB[b].append(it)
                        TBS[b].append(TYE[-1])                      # 进入缓冲区的时间 = ASC释放集装箱到支架上的结束时刻

                        ########****** 任务分配到任务量最小且空闲的车辆 *****#######
                        if self.stateDecide(KSS) == True:
                            VX = []
                            EX = list(KSS.values())
                            [VX.append(i + 1) for i in range(len(EX)) if EX[i] == False]  # 查找空闲的 AGV
                            Vcount = [len(V[v]) for v in VX]
                            k = VX[Vcount.index(min(Vcount))]               # 任务分配给当前任务量最小的空闲的AGV
                            ### k = list(KSS.values()).index(False) + 1     # 任务随机分配给          空闲的AGV
                            V[k].append(it)     # 把任务分配给该空闲车辆
                            KSS[k] = True       # 更改车辆为占用状态
                            ## AGV开始作业任务it的时间 = max{任务进入直接交互位的时间, 上一个任务的结束时间 + 相邻两任务的travel time}
                            TVS[k].append(max(TBS[b][-1], TVE[k][-1] + TK[V[k][-2]][it]))
                            TBE[b].append(TVS[k][-1])       # 任务离开间接交互位的时间
                            BSS[b] = False
                            TVE[k].append(TVS[k][-1] + TTK[it] + AGV['TQ'])
                            KSS[k] = False

            else:
                    hangingSet.append(it)
                    sequence += hangingSet # 悬挂任务返回调度池 sequence


        # print('Q: ', len(Q) - 1, Q)
        # print('V: ', V)
        # print(TVS)
        # print(TVE)
        # print(TYS)
        # print(TYE)
        # print('hangingSet: ', hangingSet)

        makes = []
        for k in K:
            makes.append(TVE[k][-1])
        makes.append(TYE[-1])
        # print(makes)
        makespan = max(makes)




        return Q, makespan, cpu, TVS, TVE, TAS, TAE, TBS, TBE, TYS, TYE, V, Q, SA, SB

    def stateDecide(self, DICT): # 可用资源判断
        A = list(DICT.values())
        if A.count(False) != 0:
            return True
        else:
            return False


class MyProblem(ea.Problem):
    def __init__(self, lenQ, flexibleASC, n):
        name = 'flexibleASC'    # name=函数名称
        M = 1                   # 目标维数=1，单目标
        maxormins = [1]         # maxormins=1：最小化；-1：最大化
        Dim = lenQ              # Dim=决策变量维数
        varTypes = [1] * Dim    # varTypes=0 [变量是连续的]；=1[离散的]

        lb = [1] * Dim          # 决策变量下界,上界
        ub = [n] * Dim
        lbin = [1] * Dim        # 决策变量下边界,上边界
        ubin = [1] * Dim

        # 调用父类构造方法完成实例化
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)
        self.flexibleASC = flexibleASC
        self.s = time.time()
        self.g = 0
        self.GAVALUE = []   # 每次迭代的最小值
        self.AVEVALUE = []  # 每次迭代的平均值
        self.mdata = {}

        ###目标函数
    def aimFunc(self, pop):     ## 解码
        x = pop.Phen    #种群的表现型矩阵
        ObjV = []

        for i in range(x.shape[0]):
            Q, makespan, cpu, TVS, TVE, TAS, TAE, TBS, TBE, TYS, TYE, V, Q, SA, SB = self.flexibleASC.assignMent(x[i, :])
            ObjV.append(makespan)


        # 用来定量描述违反约束条件程度的矩阵, 每行对应一个个体, 每列对应一个约束。无约束条件时, CV为None
        # pop.CV = np.hstack([x[:, [0]]-x[:, [1]]])
        pop.CV = None
        pop.ObjV = np.array([ObjV]).T  # 把求得的目标函数值赋值给种群pop的ObjV
        self.g += 1
        self.AVEVALUE.append(np.average((ObjV)))

        # print('E= %e' % np.min(ObjV), '\tTime=', time.time() - self.s, '\tGen=', self.g)
        # print('CV: ', len(pop.CV))
        # print('表现型矩阵： ', x)

        # ObjV包含m个目标值, 其中m为种群数量
        self.GAVALUE.append(np.min(ObjV))
        self.outData = [TVS, TVE, TAS, TAE, TBS, TBE, TYS, TYE, V, Q, SA, SB]

def geat(N, NK, NA, NB, NAME, AGV, ASC, popnum, iternum, pc, pm):
    flexibleASC = flexibleSchedulingASC()
    flexibleASC.dataGeneration(N, NK, NA, NB, NAME, AGV, ASC)

    SolsPop = []        # 解集合 by variables pm and pc
    Solution = []
    BestSol = 99999
    DATAOUTPUT = {}

    PC, PM = 0, 0   # 初始化参数
    MaxSol, AveSol, Computime = 0, 0, 0
    for c in range(len(pc)):
        for m in range(len(pm)):
            ### 实例化问题对象
            problem = MyProblem(flexibleASC.lenQ, flexibleASC, N)

            ###种群设置
            Encoding = 'P' # 编码方式
            # Encoding = 'RI' # 编码方式
            # BG，二进制 / 格雷码；
            # RI，实整数编码，即实数和整数的混合编码；
            # P，排列编码，即染色体每一位的元素都是互异的。
            NIND = popnum   #种群规模

            # 调用创建区域描述器
            Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges, problem.borders)

            # 实例化种群对象
            population = ea.Population(Encoding, Field, NIND)

            ###算法参数设置
            # 实例化一个算法模版对象
            # myAlgorithm = ea.soea_SGA_templet(problem, population)        # 最简单、最经典的遗传算法模板
            # myAlgorithm = ea.soea_EGA_templet(problem, population)        # 精英保留的遗传算法模板
            # myAlgorithm = ea.soea_SEGA_templet(problem, population)         # 增强精英保留的遗传算法模板
            myAlgorithm = ea.soea_GGAP_SGA_templet(problem, population)   # 带代沟的简单遗传算法模板
            # myAlgorithm = ea.soea_studGA_templet(problem, population)       # 种马遗传算法模板

            myAlgorithm.MAXGEN = iternum            # 最大进化代数
            myAlgorithm.recOper.XOVR = pc[c]    # 重组概率
            myAlgorithm.mutOper.Pm = pm[m]      # 变异概率
            myAlgorithm.drawing = 0             # 遗传算法求解收敛图

            Lind = int(np.sum(Field[0, :]))                     # 计算编码后的染色体长度
            obj_trace = np.zeros((myAlgorithm.MAXGEN, 2))       # 定义目标函数值记录器
            var_trace = np.zeros((myAlgorithm.MAXGEN, Lind))    # 定义染色体记录器，记录每一代最优个体的染色体

            ### 调用算法模版进行种群进化
            # 执行算法模版
            [BestIndi, population] = myAlgorithm.run()      # go to function: aimFun(pop) 执行解码操作
            # [population, BestIndi, var_trace] = myAlgorithm.run()  # 执行算法模板 # 输出结果

            BestIndi.save()
            # obj_trace.save()

            print('评价次数：%s' % myAlgorithm.evalsNum)
            print('时间已过 %s 秒' % myAlgorithm.passTime)

            TempBest = BestIndi.ObjV[0][0]
            MaxSol = np.max(problem.GAVALUE)
            AveSol = np.average(problem.GAVALUE)


            # print(problem.GAVALUE)      # 每一代的最小值
            # print(problem.AVEVALUE)     # 平均值

            #### 写入 最小值和平均值到EXCEL中 ########
            # workbook = xlwt.Workbook()
            # worksheet1 = workbook.add_sheet('minave')
            # worksheet1.write(0, 0, 'min')
            # worksheet1.write(0, 1, 'ave')
            # for i in range(len(problem.GAVALUE)):
            #     worksheet1.write(i + 1, 0, int(problem.GAVALUE[i]))
            #     worksheet1.write(i + 1, 1, int(problem.AVEVALUE[i]))
            # fnSolution = 'D:\\yzwang\\PROBLEM\\AGV_Scheduling\\AGV_ASC-Buffer\\data\\solution\\'
            # fnName = 'MinAveValue2.xls'
            # workbook.save(fnSolution + fnName)

            OUTD = {}
            if TempBest < BestSol:
                Solution = []
                BestSol = TempBest
                OUTD = problem.outData
                PM = pm[m]
                PC = pc[c]
                for i in range(BestIndi.Phen.shape[1]):
                    Solution.append(BestIndi.Phen[0, i])

            DATAOUTPUT[(c, m)] = [TempBest, MaxSol, AveSol]
            SolsPop.append({(pc[c], pm[m], np.round(myAlgorithm.passTime, 2)): [np.round(TempBest, 2), np.round(MaxSol, 2), np.round(AveSol, 2)]})

            if BestIndi.sizes != 0:
                print('最优的目标函数值为：%s' % BestSol)
                # print('最优的控制变量值为：')
                # print(Solution)
                # print(OUTD)
            else:
                print('没找到可行解')

            # print('最小值, 最大值, 平均值: ')
            # print(np.min(problem.GAVALUE), MaxSol, AveSol)
            Computime = np.round(myAlgorithm.passTime, 2)
    print('{(pc, pm, cpu): [最小值, 最大值, 平均值]}')
    for ee in SolsPop:
        print(ee, end='\n')
    # print('BestSolution: ', {(PC, PM): [BestSol, MaxSol, AveSol]})
    # print('Computime: ', Computime)

    # ASC.writeFigureMatlab(DATAOUTPUT, num, pc, pm)      # 写进excel, 送入matlab中作图（等高线图）

    # return Solution, BestSol, MaxSol, AveSol, SolsPop, Computime, OUTD
    return Solution, BestSol, MaxSol, AveSol, SolsPop, Computime





In [4]:
def flexibleSchedulingGA1():

    ###*** 码头作业设备参数设置
    AGV = {'SPD': 4, 'BK': 10, 'TQ': 15}    # AGV参数：水平移动速度、支架上抬/放箱时间、岸桥下作业时间
    ASC = {'SPD': 6, 'BY': 5, 'TY': 10}     # ASC参数：水平移动速度、抓取/释放箱时间、起/落吊时间

    ###*** 遗传算法参数设置
    popnum = 40          # 种群数量
    iternum = 1000       # 进化代数
    # sp1, sp2 = 0.1, 0.1
    # pc = np.arange(0.1, 1.0, sp1)    # PC的取值
    # pm = np.arange(0.1, 1.0, sp1)    # PM的取值
    # pc = [0.7, 0.8]
    # pm = [0.3, 0.4]
    pc = [0.8]
    pm = [0.4]

    ###*** 算例参数设置
    NAME = "testing"
    NM = [20]  # 任务数
    # NK = 4  # AGV数量
    MK = [4]    # AGV数量集合
    # NAB = {1: (2, 3), 2: (3, 3), 3: (4, 3), 4: (5, 3), 5: (6, 3)}  # (硬,软）交互位数量
    NAB = {1:(3, 2), 2: (3, 3), 3: (3, 4), 4: (3, 5), 5:(3, 6)}   # (硬,软）交互位数量

    # # ##*** 1 单次计算  ***###
    # ab = 1  # 交互位选择
    # NA = NAB[ab][0]
    # NB = NAB[ab][1]
    SOLK = {}
    Computime = 0


    for N in NM:
        for NK in MK:
            for ab in range(1, 6):
                NA = NAB[ab][0]
                NB = NAB[ab][1]
                SOLLmin, SOLLmax, SOLLave = [], [], []
                OUTDATA = {}
                for i in range(10):  # 重复计算n次
                    Solution, BestSol, MaxSol, AveSol, SolsPop, Computime = geat(N, NK, NA, NB, NAME, AGV, ASC, popnum, iternum, pc, pm)
                    SOLLmin.append(BestSol) #
                    SOLLmax.append(MaxSol)
                    SOLLave.append(AveSol)
                    # OUTDATA[i] = OUTD
                SOLK[(N, NK, NAB[ab])] = (min(SOLLmin), min(SOLLmax), min(SOLLave), Computime) # makespan取多次的计算的最小值 min(SOLL)

    # print("N = {:2} | K = {:} | flexible = {} | makespan {:7.2f} units | {:7.3f} secs".format(N, NK, NAB[ab], BestSol, Computime))
    # print(SOLL)
    # print(ASC['SPD'], min(SOLL))
    # print("N = {:2} | K = {:} | flexible = {} | makespan {:7.2f} units".format(N, NK, NAB[ab], min(SOLL)))

    for ee in SOLK.keys():
        print(ee, SOLK[ee], end='\n')

    # for er in OUTDATA.keys():
    #     print(er, OUTDATA[er], end='\n')
flexibleSchedulingGA1()

gen|  eval  |    f_opt    |    f_max    |    f_avg    |    f_min    |    f_std    
----------------------------------------------------------------------------------
 0 |   40   | 3.38650E+03 | 4.19425E+03 | 3.85639E+03 | 3.38650E+03 | 1.73313E+02 
 1 |   80   | 3.38650E+03 | 4.06250E+03 | 3.75698E+03 | 3.38650E+03 | 1.85570E+02 
 2 |  120   | 3.33425E+03 | 3.96100E+03 | 3.66056E+03 | 3.33425E+03 | 1.66200E+02 
 3 |  160   | 3.33425E+03 | 3.94225E+03 | 3.64919E+03 | 3.33425E+03 | 1.50027E+02 
 4 |  200   | 3.33425E+03 | 3.84900E+03 | 3.58534E+03 | 3.33425E+03 | 1.57794E+02 
 5 |  240   | 3.33425E+03 | 3.90925E+03 | 3.56624E+03 | 3.33425E+03 | 1.82658E+02 
 6 |  280   | 3.23925E+03 | 3.96600E+03 | 3.57339E+03 | 3.23925E+03 | 2.00046E+02 
 7 |  320   | 3.23925E+03 | 3.78400E+03 | 3.53703E+03 | 3.23925E+03 | 1.56536E+02 
 8 |  360   | 3.23425E+03 | 3.82900E+03 | 3.51959E+03 | 3.23425E+03 | 1.59726E+02 
 9 |  400   | 3.23425E+03 | 3.84900E+03 | 3.53536E+03 | 3.23425E+03 | 1.89149E+02 
 10|

109|  4400  | 2.98475E+03 | 3.49525E+03 | 3.14211E+03 | 2.98475E+03 | 1.62361E+02 
110|  4440  | 2.98475E+03 | 3.52725E+03 | 3.12179E+03 | 2.98475E+03 | 1.67561E+02 
111|  4480  | 2.98475E+03 | 3.47075E+03 | 3.13245E+03 | 2.98475E+03 | 1.41950E+02 
112|  4520  | 2.97925E+03 | 3.55175E+03 | 3.14637E+03 | 2.97925E+03 | 1.69540E+02 
113|  4560  | 2.97925E+03 | 3.58850E+03 | 3.19434E+03 | 2.97925E+03 | 1.77974E+02 
114|  4600  | 2.97925E+03 | 3.57100E+03 | 3.19119E+03 | 2.97925E+03 | 1.66151E+02 
115|  4640  | 2.97925E+03 | 3.31125E+03 | 3.08146E+03 | 2.97925E+03 | 1.13244E+02 
116|  4680  | 2.97925E+03 | 3.50275E+03 | 3.10250E+03 | 2.97925E+03 | 1.51918E+02 
117|  4720  | 2.97925E+03 | 3.59375E+03 | 3.14744E+03 | 2.97925E+03 | 1.72392E+02 
118|  4760  | 2.97925E+03 | 3.58125E+03 | 3.20631E+03 | 2.97925E+03 | 1.75545E+02 
119|  4800  | 2.97925E+03 | 3.66525E+03 | 3.18983E+03 | 2.97925E+03 | 1.88394E+02 
120|  4840  | 2.97925E+03 | 3.41975E+03 | 3.18376E+03 | 2.97925E+03 | 1.30349E+02 
121|

211|  8480  | 2.97925E+03 | 3.40725E+03 | 3.12781E+03 | 2.97925E+03 | 1.26490E+02 
212|  8520  | 2.97925E+03 | 3.55025E+03 | 3.13496E+03 | 2.97925E+03 | 1.63445E+02 
213|  8560  | 2.97925E+03 | 3.50625E+03 | 3.15378E+03 | 2.97925E+03 | 1.45683E+02 
214|  8600  | 2.97925E+03 | 3.49925E+03 | 3.13151E+03 | 2.97925E+03 | 1.25442E+02 
215|  8640  | 2.97925E+03 | 3.46925E+03 | 3.17247E+03 | 2.97925E+03 | 1.39990E+02 
216|  8680  | 2.97925E+03 | 3.56650E+03 | 3.17512E+03 | 2.97925E+03 | 1.58141E+02 
217|  8720  | 2.97925E+03 | 3.42125E+03 | 3.16524E+03 | 2.97925E+03 | 1.24249E+02 
218|  8760  | 2.97925E+03 | 3.54975E+03 | 3.17744E+03 | 2.97925E+03 | 1.70014E+02 
219|  8800  | 2.97925E+03 | 3.51650E+03 | 3.13383E+03 | 2.97925E+03 | 1.44448E+02 
220|  8840  | 2.97925E+03 | 3.48825E+03 | 3.17753E+03 | 2.97925E+03 | 1.54532E+02 
221|  8880  | 2.97925E+03 | 3.46725E+03 | 3.17877E+03 | 2.97925E+03 | 1.44665E+02 
222|  8920  | 2.97925E+03 | 3.44175E+03 | 3.15618E+03 | 2.97925E+03 | 1.24836E+02 
223|

322| 12920  | 2.97925E+03 | 3.47625E+03 | 3.16076E+03 | 2.97925E+03 | 1.74158E+02 
323| 12960  | 2.97925E+03 | 3.50325E+03 | 3.15232E+03 | 2.97925E+03 | 1.61125E+02 
324| 13000  | 2.97925E+03 | 3.41525E+03 | 3.12344E+03 | 2.97925E+03 | 1.40525E+02 
325| 13040  | 2.97925E+03 | 3.47025E+03 | 3.11679E+03 | 2.97925E+03 | 1.45207E+02 
326| 13080  | 2.97925E+03 | 3.41825E+03 | 3.10241E+03 | 2.97925E+03 | 1.35221E+02 
327| 13120  | 2.97925E+03 | 3.54175E+03 | 3.14014E+03 | 2.97925E+03 | 1.65775E+02 
328| 13160  | 2.97925E+03 | 3.52225E+03 | 3.13757E+03 | 2.97925E+03 | 1.60372E+02 
329| 13200  | 2.97925E+03 | 3.32425E+03 | 3.10073E+03 | 2.97925E+03 | 1.20793E+02 
330| 13240  | 2.97925E+03 | 3.51275E+03 | 3.14035E+03 | 2.97925E+03 | 1.62044E+02 
331| 13280  | 2.97925E+03 | 3.40925E+03 | 3.15493E+03 | 2.97925E+03 | 1.44300E+02 
332| 13320  | 2.97925E+03 | 3.44625E+03 | 3.10124E+03 | 2.97925E+03 | 1.34020E+02 
333| 13360  | 2.97925E+03 | 3.44175E+03 | 3.09450E+03 | 2.97925E+03 | 1.38982E+02 
334|

420| 16840  | 2.97925E+03 | 3.62125E+03 | 3.19249E+03 | 2.97925E+03 | 1.82737E+02 
421| 16880  | 2.97925E+03 | 3.55275E+03 | 3.19933E+03 | 2.97925E+03 | 1.77659E+02 
422| 16920  | 2.97925E+03 | 3.57625E+03 | 3.21559E+03 | 2.97925E+03 | 1.94610E+02 
423| 16960  | 2.97925E+03 | 3.62775E+03 | 3.19781E+03 | 2.97925E+03 | 2.06666E+02 
424| 17000  | 2.97925E+03 | 3.63625E+03 | 3.17314E+03 | 2.97925E+03 | 2.19210E+02 
425| 17040  | 2.97925E+03 | 3.33275E+03 | 3.09400E+03 | 2.97925E+03 | 1.15639E+02 
426| 17080  | 2.97925E+03 | 3.51925E+03 | 3.14269E+03 | 2.97925E+03 | 1.78824E+02 
427| 17120  | 2.97925E+03 | 3.36775E+03 | 3.08171E+03 | 2.97925E+03 | 1.23326E+02 
428| 17160  | 2.97925E+03 | 3.49625E+03 | 3.15084E+03 | 2.97925E+03 | 1.92004E+02 
429| 17200  | 2.97925E+03 | 3.58375E+03 | 3.16620E+03 | 2.97925E+03 | 1.99794E+02 
430| 17240  | 2.97925E+03 | 3.44175E+03 | 3.13576E+03 | 2.97925E+03 | 1.52365E+02 
431| 17280  | 2.97925E+03 | 3.46975E+03 | 3.12146E+03 | 2.97925E+03 | 1.54626E+02 
432|

...

551| 22080  | 2.93550E+03 | 3.71000E+03 | 3.19779E+03 | 2.93550E+03 | 2.32326E+02 
552| 22120  | 2.93550E+03 | 3.61550E+03 | 3.17049E+03 | 2.93550E+03 | 2.16369E+02 
553| 22160  | 2.93550E+03 | 3.55700E+03 | 3.16570E+03 | 2.93550E+03 | 1.87758E+02 
554| 22200  | 2.93550E+03 | 3.66150E+03 | 3.20316E+03 | 2.93550E+03 | 2.21773E+02 
555| 22240  | 2.93550E+03 | 3.41825E+03 | 3.13746E+03 | 2.93550E+03 | 1.56528E+02 
556| 22280  | 2.93550E+03 | 3.72550E+03 | 3.15020E+03 | 2.93550E+03 | 2.12889E+02 
557| 22320  | 2.93550E+03 | 3.56600E+03 | 3.17122E+03 | 2.93550E+03 | 1.91178E+02 
558| 22360  | 2.93550E+03 | 3.72100E+03 | 3.20272E+03 | 2.93550E+03 | 2.31752E+02 
559| 22400  | 2.93550E+03 | 3.71150E+03 | 3.23673E+03 | 2.93550E+03 | 2.50367E+02 
560| 22440  | 2.93550E+03 | 3.67000E+03 | 3.15308E+03 | 2.93550E+03 | 2.26307E+02 
561| 22480  | 2.93550E+03 | 3.52200E+03 | 3.09304E+03 | 2.93550E+03 | 1.66230E+02 
562| 22520  | 2.93550E+03 | 3.70350E+03 | 3.11919E+03 | 2.93550E+03 | 2.03010E+02 
563|

651| 26080  | 2.86400E+03 | 3.63025E+03 | 3.11202E+03 | 2.86400E+03 | 2.20218E+02 
652| 26120  | 2.86400E+03 | 3.69250E+03 | 3.12885E+03 | 2.86400E+03 | 2.16941E+02 
653| 26160  | 2.86400E+03 | 3.50625E+03 | 3.11722E+03 | 2.86400E+03 | 2.05837E+02 
654| 26200  | 2.86400E+03 | 3.45700E+03 | 3.10107E+03 | 2.86400E+03 | 1.86526E+02 
655| 26240  | 2.86400E+03 | 3.34550E+03 | 3.04332E+03 | 2.86400E+03 | 1.60919E+02 
656| 26280  | 2.86400E+03 | 3.24825E+03 | 3.01355E+03 | 2.86400E+03 | 1.34934E+02 
657| 26320  | 2.86400E+03 | 3.36700E+03 | 3.00268E+03 | 2.86400E+03 | 1.52444E+02 
658| 26360  | 2.86400E+03 | 3.33150E+03 | 2.99675E+03 | 2.86400E+03 | 1.52435E+02 
659| 26400  | 2.86400E+03 | 3.25750E+03 | 2.92401E+03 | 2.86400E+03 | 1.10042E+02 
660| 26440  | 2.86400E+03 | 3.53425E+03 | 3.00491E+03 | 2.86400E+03 | 1.73200E+02 
661| 26480  | 2.86400E+03 | 3.68100E+03 | 3.07296E+03 | 2.86400E+03 | 2.53071E+02 
662| 26520  | 2.86400E+03 | 3.38400E+03 | 3.01138E+03 | 2.86400E+03 | 1.67901E+02 
663|

763| 30560  | 2.86400E+03 | 3.40900E+03 | 2.98749E+03 | 2.86400E+03 | 1.73454E+02 
764| 30600  | 2.86400E+03 | 3.27300E+03 | 2.98002E+03 | 2.86400E+03 | 1.42153E+02 
765| 30640  | 2.86400E+03 | 3.47950E+03 | 3.00911E+03 | 2.86400E+03 | 2.02482E+02 
766| 30680  | 2.86400E+03 | 3.53150E+03 | 2.97347E+03 | 2.86400E+03 | 1.68542E+02 
767| 30720  | 2.86400E+03 | 3.53400E+03 | 3.01949E+03 | 2.86400E+03 | 2.20735E+02 
768| 30760  | 2.86400E+03 | 3.42425E+03 | 3.00305E+03 | 2.86400E+03 | 1.61927E+02 
769| 30800  | 2.86400E+03 | 3.35900E+03 | 3.03367E+03 | 2.86400E+03 | 1.76993E+02 
770| 30840  | 2.86400E+03 | 3.35100E+03 | 2.96327E+03 | 2.86400E+03 | 1.33759E+02 
771| 30880  | 2.86400E+03 | 3.43350E+03 | 2.97919E+03 | 2.86400E+03 | 1.66849E+02 
772| 30920  | 2.86400E+03 | 3.62550E+03 | 3.01667E+03 | 2.86400E+03 | 2.03395E+02 
773| 30960  | 2.86400E+03 | 3.38400E+03 | 3.00575E+03 | 2.86400E+03 | 1.87799E+02 
774| 31000  | 2.86400E+03 | 3.36525E+03 | 3.06212E+03 | 2.86400E+03 | 1.73036E+02 
775|

873| 34960  | 2.86400E+03 | 3.49925E+03 | 3.05664E+03 | 2.86400E+03 | 2.04204E+02 
874| 35000  | 2.86400E+03 | 3.33475E+03 | 2.97479E+03 | 2.86400E+03 | 1.34545E+02 
875| 35040  | 2.86400E+03 | 3.54300E+03 | 2.98526E+03 | 2.86400E+03 | 1.56921E+02 
876| 35080  | 2.86400E+03 | 3.41800E+03 | 2.98864E+03 | 2.86400E+03 | 1.79568E+02 
877| 35120  | 2.86400E+03 | 3.65900E+03 | 3.06670E+03 | 2.86400E+03 | 2.30483E+02 
878| 35160  | 2.86400E+03 | 3.53150E+03 | 3.09871E+03 | 2.86400E+03 | 2.20151E+02 
879| 35200  | 2.86400E+03 | 3.45200E+03 | 3.09612E+03 | 2.86400E+03 | 2.01310E+02 
880| 35240  | 2.86400E+03 | 3.47800E+03 | 3.03038E+03 | 2.86400E+03 | 2.00914E+02 
881| 35280  | 2.86400E+03 | 3.41700E+03 | 3.02561E+03 | 2.86400E+03 | 1.81816E+02 
882| 35320  | 2.86400E+03 | 3.48400E+03 | 3.00236E+03 | 2.86400E+03 | 1.82705E+02 
883| 35360  | 2.86400E+03 | 3.47950E+03 | 2.97048E+03 | 2.86400E+03 | 1.50463E+02 
884| 35400  | 2.86400E+03 | 3.43900E+03 | 3.04331E+03 | 2.86400E+03 | 1.70844E+02 
885|

985| 39440  | 2.86400E+03 | 3.55700E+03 | 3.06785E+03 | 2.86400E+03 | 2.05375E+02 
986| 39480  | 2.86400E+03 | 3.55700E+03 | 3.05947E+03 | 2.86400E+03 | 2.03042E+02 
987| 39520  | 2.86400E+03 | 3.45950E+03 | 3.06031E+03 | 2.86400E+03 | 2.03212E+02 
988| 39560  | 2.86400E+03 | 3.37700E+03 | 3.04054E+03 | 2.86400E+03 | 1.78969E+02 
989| 39600  | 2.86400E+03 | 3.33200E+03 | 3.02008E+03 | 2.86400E+03 | 1.47319E+02 
990| 39640  | 2.86400E+03 | 3.57400E+03 | 3.07429E+03 | 2.86400E+03 | 2.20279E+02 
991| 39680  | 2.86400E+03 | 3.29600E+03 | 2.99097E+03 | 2.86400E+03 | 1.43995E+02 
992| 39720  | 2.86400E+03 | 3.33900E+03 | 3.01064E+03 | 2.86400E+03 | 1.47955E+02 
993| 39760  | 2.86400E+03 | 3.58300E+03 | 3.03481E+03 | 2.86400E+03 | 1.74864E+02 
994| 39800  | 2.86400E+03 | 3.39300E+03 | 3.04621E+03 | 2.86400E+03 | 1.74591E+02 
995| 39840  | 2.86400E+03 | 3.41100E+03 | 3.03426E+03 | 2.86400E+03 | 1.64670E+02 
996| 39880  | 2.86400E+03 | 3.49650E+03 | 3.03244E+03 | 2.86400E+03 | 2.02191E+02 
997|

In [3]:
def flexibleSchedulingGA2():
    #### *** 2重复计算，对比不同软硬交互位组合下的规划结果 ***###

    ###*** 码头作业设备参数设置
    AGV = {'SPD': 4, 'BK': 10, 'TQ': 15}    # AGV参数：水平移动速度、支架上抬/放箱时间、岸桥下作业时间
    ASC = {'SPD': 6, 'BY': 5, 'TY': 10}     # ASC参数：水平移动速度、抓取/释放箱时间、起/落吊时间

    ###*** 遗传算法参数设置
    popnum = 40          # 种群数量
    iternum = 500       # 进化代数
    # sp1, sp2 = 0.1, 0.1
    # pc = np.arange(0.1, 1.0, sp1)    # PC的取值
    # pm = np.arange(0.1, 1.0, sp1)    # PM的取值
    # pc = [0.7, 0.8]
    # pm = [0.3, 0.4]
    pc = [0.8]
    pm = [0.4]

    ###*** 算例参数设置
    NAME = "testing"
    NM = [20]  # 任务数
    NAB = {1:(3, 2), 2: (3, 3), 3: (3, 4), 4: (3, 5), 5:(3, 6)}   # (硬,软）交互位数量

    workbook = xlwt.Workbook()
    worksheet = workbook.add_sheet('saveTempData')
    N = 20  # 任务数量
    NK = 4  # AGV数量

    for i in range(20):     # 计算20次，存档excel
        SOL = {}
        for ab in range(1, 6):
            NA = NAB[ab][0]
            NB = NAB[ab][1]
            Solution, BestSol, MaxSol, AveSol, SolsPop, Computime = geat(N, NK, NA, NB, NAME, AGV, ASC, popnum, iternum, pc, pm)
            SOL[(NA, NB)] = BestSol
        x = 0
        for ss in SOL:
            worksheet.write(i, x, SOL[ss])
            x += 1

    mDir = '../data/'
    mDir_solution = 'solution/'

    fnName = 'ComputiontempDataBOX200.xls'
    workbook.save(mDir + mDir_solution + fnName)



flexibleSchedulingGA2()

gen|  eval  |    f_opt    |    f_max    |    f_avg    |    f_min    |    f_std    
----------------------------------------------------------------------------------
 0 |   40   | 3.36550E+03 | 4.18625E+03 | 3.78920E+03 | 3.36550E+03 | 1.89534E+02 
 1 |   80   | 3.36500E+03 | 3.94700E+03 | 3.69853E+03 | 3.36500E+03 | 1.55451E+02 
 2 |  120   | 3.36500E+03 | 3.93550E+03 | 3.71993E+03 | 3.36500E+03 | 1.67667E+02 
 3 |  160   | 3.33250E+03 | 3.96400E+03 | 3.68276E+03 | 3.33250E+03 | 1.81231E+02 
 4 |  200   | 3.33250E+03 | 3.86350E+03 | 3.64876E+03 | 3.33250E+03 | 1.57576E+02 
 5 |  240   | 3.33250E+03 | 3.92375E+03 | 3.63296E+03 | 3.33250E+03 | 1.95130E+02 
 6 |  280   | 3.30200E+03 | 3.89425E+03 | 3.58358E+03 | 3.30200E+03 | 1.80316E+02 
 7 |  320   | 3.27100E+03 | 3.84550E+03 | 3.55921E+03 | 3.27100E+03 | 1.60943E+02 
 8 |  360   | 3.21250E+03 | 3.90800E+03 | 3.56196E+03 | 3.21250E+03 | 1.91737E+02 
 9 |  400   | 3.18950E+03 | 3.75575E+03 | 3.47919E+03 | 3.18950E+03 | 1.63128E+02 
 10|

 95|  3840  | 2.99250E+03 | 3.30750E+03 | 3.07007E+03 | 2.99250E+03 | 1.00487E+02 
 96|  3880  | 2.99250E+03 | 3.45250E+03 | 3.10709E+03 | 2.99250E+03 | 1.43505E+02 
 97|  3920  | 2.99250E+03 | 3.33850E+03 | 3.09113E+03 | 2.99250E+03 | 1.17343E+02 
 98|  3960  | 2.96800E+03 | 3.48650E+03 | 3.10968E+03 | 2.96800E+03 | 1.63937E+02 
 99|  4000  | 2.96800E+03 | 3.52300E+03 | 3.11261E+03 | 2.96800E+03 | 1.62959E+02 
100|  4040  | 2.96800E+03 | 3.48050E+03 | 3.12590E+03 | 2.96800E+03 | 1.49428E+02 
101|  4080  | 2.96800E+03 | 3.49400E+03 | 3.15396E+03 | 2.96800E+03 | 1.68943E+02 
102|  4120  | 2.96800E+03 | 3.54550E+03 | 3.20482E+03 | 2.96800E+03 | 2.03831E+02 
103|  4160  | 2.96800E+03 | 3.51700E+03 | 3.10031E+03 | 2.96800E+03 | 1.62986E+02 
104|  4200  | 2.96800E+03 | 3.42500E+03 | 3.08963E+03 | 2.96800E+03 | 1.47148E+02 
105|  4240  | 2.96800E+03 | 3.42500E+03 | 3.07394E+03 | 2.96800E+03 | 1.37427E+02 
106|  4280  | 2.96800E+03 | 3.42500E+03 | 3.07131E+03 | 2.96800E+03 | 1.43674E+02 
107|

202|  8120  | 2.91400E+03 | 3.72100E+03 | 3.17222E+03 | 2.91400E+03 | 2.35068E+02 
203|  8160  | 2.89250E+03 | 3.60500E+03 | 3.13009E+03 | 2.89250E+03 | 2.16134E+02 
204|  8200  | 2.89250E+03 | 3.78725E+03 | 3.17864E+03 | 2.89250E+03 | 2.61463E+02 
205|  8240  | 2.89250E+03 | 3.65550E+03 | 3.23108E+03 | 2.89250E+03 | 2.30773E+02 
206|  8280  | 2.89250E+03 | 3.54000E+03 | 3.23643E+03 | 2.89250E+03 | 1.98759E+02 
207|  8320  | 2.89250E+03 | 3.77950E+03 | 3.25142E+03 | 2.89250E+03 | 2.43776E+02 
208|  8360  | 2.89250E+03 | 3.68250E+03 | 3.27539E+03 | 2.89250E+03 | 2.46023E+02 
209|  8400  | 2.89250E+03 | 3.64250E+03 | 3.27309E+03 | 2.89250E+03 | 2.01740E+02 
210|  8440  | 2.89250E+03 | 3.61000E+03 | 3.29674E+03 | 2.89250E+03 | 2.02485E+02 
211|  8480  | 2.89250E+03 | 3.77850E+03 | 3.33491E+03 | 2.89250E+03 | 2.30553E+02 
212|  8520  | 2.89250E+03 | 3.57675E+03 | 3.29948E+03 | 2.89250E+03 | 2.01295E+02 
213|  8560  | 2.89250E+03 | 3.68250E+03 | 3.31649E+03 | 2.89250E+03 | 2.79650E+02 
214|

309| 12400  | 2.89250E+03 | 3.49850E+03 | 3.08224E+03 | 2.89250E+03 | 1.92074E+02 
310| 12440  | 2.89250E+03 | 3.66700E+03 | 3.12009E+03 | 2.89250E+03 | 2.70884E+02 
311| 12480  | 2.89250E+03 | 3.58150E+03 | 3.08578E+03 | 2.89250E+03 | 2.13619E+02 
312| 12520  | 2.89250E+03 | 3.37950E+03 | 3.04454E+03 | 2.89250E+03 | 1.58432E+02 
313| 12560  | 2.89250E+03 | 3.62050E+03 | 3.12366E+03 | 2.89250E+03 | 2.32826E+02 
314| 12600  | 2.89250E+03 | 3.45200E+03 | 3.08434E+03 | 2.89250E+03 | 1.76565E+02 
315| 12640  | 2.89250E+03 | 3.67750E+03 | 3.10468E+03 | 2.89250E+03 | 2.25070E+02 
316| 12680  | 2.89250E+03 | 3.76450E+03 | 3.12595E+03 | 2.89250E+03 | 2.42349E+02 
317| 12720  | 2.89250E+03 | 3.53650E+03 | 3.12032E+03 | 2.89250E+03 | 2.16032E+02 
318| 12760  | 2.89250E+03 | 3.55900E+03 | 3.13234E+03 | 2.89250E+03 | 2.24543E+02 
319| 12800  | 2.89250E+03 | 3.58250E+03 | 3.25762E+03 | 2.89250E+03 | 2.26826E+02 
320| 12840  | 2.89250E+03 | 3.52200E+03 | 3.14038E+03 | 2.89250E+03 | 2.05950E+02 
321|

424| 17000  | 2.89250E+03 | 3.62650E+03 | 3.10146E+03 | 2.89250E+03 | 2.43055E+02 
425| 17040  | 2.89250E+03 | 3.46200E+03 | 3.06758E+03 | 2.89250E+03 | 1.85013E+02 
426| 17080  | 2.89250E+03 | 3.53650E+03 | 3.05872E+03 | 2.89250E+03 | 1.81598E+02 
427| 17120  | 2.89250E+03 | 3.58150E+03 | 3.12403E+03 | 2.89250E+03 | 2.14581E+02 
428| 17160  | 2.89250E+03 | 3.58675E+03 | 3.15046E+03 | 2.89250E+03 | 2.14801E+02 
429| 17200  | 2.89250E+03 | 3.50650E+03 | 3.10838E+03 | 2.89250E+03 | 1.95781E+02 
430| 17240  | 2.89250E+03 | 3.32550E+03 | 3.01694E+03 | 2.89250E+03 | 1.29481E+02 
431| 17280  | 2.89250E+03 | 3.59100E+03 | 3.07771E+03 | 2.89250E+03 | 2.00792E+02 
432| 17320  | 2.89250E+03 | 3.57300E+03 | 3.10953E+03 | 2.89250E+03 | 2.01288E+02 
433| 17360  | 2.89250E+03 | 3.64350E+03 | 3.15721E+03 | 2.89250E+03 | 2.47790E+02 
434| 17400  | 2.89250E+03 | 3.68200E+03 | 3.19846E+03 | 2.89250E+03 | 2.54705E+02 
435| 17440  | 2.89250E+03 | 3.59750E+03 | 3.15904E+03 | 2.89250E+03 | 2.24930E+02 
436|

...

 25|  1040  | 2.90700E+03 | 3.59800E+03 | 3.10612E+03 | 2.90700E+03 | 1.91579E+02 
 26|  1080  | 2.90700E+03 | 3.39550E+03 | 3.05606E+03 | 2.90700E+03 | 1.20499E+02 
 27|  1120  | 2.90700E+03 | 3.49350E+03 | 3.05709E+03 | 2.90700E+03 | 1.53738E+02 
 28|  1160  | 2.90700E+03 | 3.43750E+03 | 3.07389E+03 | 2.90700E+03 | 1.52886E+02 
 29|  1200  | 2.90700E+03 | 3.46750E+03 | 3.05271E+03 | 2.90700E+03 | 1.63219E+02 
 30|  1240  | 2.90700E+03 | 3.38800E+03 | 3.02845E+03 | 2.90700E+03 | 1.44825E+02 
 31|  1280  | 2.90700E+03 | 3.43550E+03 | 3.06820E+03 | 2.90700E+03 | 1.82667E+02 
 32|  1320  | 2.90700E+03 | 3.54550E+03 | 3.07623E+03 | 2.90700E+03 | 1.80295E+02 
 33|  1360  | 2.90700E+03 | 3.49950E+03 | 3.08009E+03 | 2.90700E+03 | 1.79921E+02 
 34|  1400  | 2.90700E+03 | 3.25550E+03 | 3.01328E+03 | 2.90700E+03 | 1.13484E+02 
 35|  1440  | 2.90700E+03 | 3.41000E+03 | 2.98829E+03 | 2.90700E+03 | 1.28634E+02 
 36|  1480  | 2.90700E+03 | 3.39800E+03 | 3.03162E+03 | 2.90700E+03 | 1.48226E+02 
 37|

132|  5320  | 2.90300E+03 | 3.62550E+03 | 3.20038E+03 | 2.90300E+03 | 2.33888E+02 
133|  5360  | 2.90300E+03 | 3.56825E+03 | 3.10529E+03 | 2.90300E+03 | 1.59457E+02 
134|  5400  | 2.90300E+03 | 3.45700E+03 | 3.09675E+03 | 2.90300E+03 | 1.48675E+02 
135|  5440  | 2.90300E+03 | 3.53800E+03 | 3.14578E+03 | 2.90300E+03 | 2.00411E+02 
136|  5480  | 2.90300E+03 | 3.54200E+03 | 3.09641E+03 | 2.90300E+03 | 1.89628E+02 
137|  5520  | 2.90300E+03 | 3.61850E+03 | 3.11648E+03 | 2.90300E+03 | 1.75678E+02 
138|  5560  | 2.90300E+03 | 3.66200E+03 | 3.15351E+03 | 2.90300E+03 | 2.36746E+02 
139|  5600  | 2.90300E+03 | 3.55550E+03 | 3.12379E+03 | 2.90300E+03 | 1.95223E+02 
140|  5640  | 2.90300E+03 | 3.72675E+03 | 3.12668E+03 | 2.90300E+03 | 2.29822E+02 
141|  5680  | 2.90300E+03 | 3.57600E+03 | 3.07229E+03 | 2.90300E+03 | 1.85351E+02 
142|  5720  | 2.90300E+03 | 3.41100E+03 | 3.04404E+03 | 2.90300E+03 | 1.78883E+02 
143|  5760  | 2.90300E+03 | 3.53400E+03 | 3.08474E+03 | 2.90300E+03 | 1.92101E+02 
144|

230|  9240  | 2.87100E+03 | 3.28450E+03 | 3.06011E+03 | 2.87100E+03 | 1.36855E+02 
231|  9280  | 2.87100E+03 | 3.52300E+03 | 3.05653E+03 | 2.87100E+03 | 1.90334E+02 
232|  9320  | 2.87100E+03 | 3.61350E+03 | 3.08370E+03 | 2.87100E+03 | 2.22809E+02 
233|  9360  | 2.87100E+03 | 3.55600E+03 | 3.03602E+03 | 2.87100E+03 | 1.73779E+02 
234|  9400  | 2.87100E+03 | 3.43100E+03 | 3.02443E+03 | 2.87100E+03 | 1.61372E+02 
235|  9440  | 2.87100E+03 | 3.53825E+03 | 3.05179E+03 | 2.87100E+03 | 1.85348E+02 
236|  9480  | 2.87100E+03 | 3.32000E+03 | 3.03864E+03 | 2.87100E+03 | 1.59394E+02 
237|  9520  | 2.87100E+03 | 3.64250E+03 | 3.04728E+03 | 2.87100E+03 | 2.37577E+02 
238|  9560  | 2.87100E+03 | 3.59100E+03 | 3.05426E+03 | 2.87100E+03 | 2.33740E+02 
239|  9600  | 2.87100E+03 | 3.58750E+03 | 3.05647E+03 | 2.87100E+03 | 2.05850E+02 
240|  9640  | 2.87100E+03 | 3.52700E+03 | 3.02761E+03 | 2.87100E+03 | 1.93661E+02 
241|  9680  | 2.87100E+03 | 3.57600E+03 | 3.07455E+03 | 2.87100E+03 | 2.07670E+02 
242|

341| 13680  | 2.87100E+03 | 3.59900E+03 | 2.97627E+03 | 2.87100E+03 | 1.63287E+02 
342| 13720  | 2.87100E+03 | 3.41750E+03 | 2.99089E+03 | 2.87100E+03 | 1.57903E+02 
343| 13760  | 2.87100E+03 | 3.57200E+03 | 3.07253E+03 | 2.87100E+03 | 2.06952E+02 
344| 13800  | 2.87100E+03 | 3.54500E+03 | 3.05382E+03 | 2.87100E+03 | 2.09340E+02 
345| 13840  | 2.87100E+03 | 3.53450E+03 | 3.08836E+03 | 2.87100E+03 | 2.05851E+02 
346| 13880  | 2.87100E+03 | 3.65800E+03 | 3.06526E+03 | 2.87100E+03 | 2.01344E+02 
347| 13920  | 2.87100E+03 | 3.70925E+03 | 3.13573E+03 | 2.87100E+03 | 2.51933E+02 
348| 13960  | 2.87100E+03 | 3.63550E+03 | 3.08096E+03 | 2.87100E+03 | 2.20872E+02 
349| 14000  | 2.87100E+03 | 3.62950E+03 | 3.18103E+03 | 2.87100E+03 | 2.48890E+02 
350| 14040  | 2.87100E+03 | 3.66400E+03 | 3.03362E+03 | 2.87100E+03 | 1.98817E+02 
351| 14080  | 2.87100E+03 | 3.35800E+03 | 3.01597E+03 | 2.87100E+03 | 1.55389E+02 
352| 14120  | 2.87100E+03 | 3.34400E+03 | 3.03328E+03 | 2.87100E+03 | 1.60535E+02 
353|

439| 17600  | 2.87100E+03 | 3.58950E+03 | 3.09629E+03 | 2.87100E+03 | 2.55557E+02 
440| 17640  | 2.87100E+03 | 3.55050E+03 | 3.04788E+03 | 2.87100E+03 | 1.92056E+02 
441| 17680  | 2.87100E+03 | 3.43950E+03 | 3.04701E+03 | 2.87100E+03 | 1.89329E+02 
442| 17720  | 2.87100E+03 | 3.53150E+03 | 3.07228E+03 | 2.87100E+03 | 2.09414E+02 
443| 17760  | 2.87100E+03 | 3.54625E+03 | 3.07951E+03 | 2.87100E+03 | 2.11662E+02 
444| 17800  | 2.87100E+03 | 3.23000E+03 | 3.03168E+03 | 2.87100E+03 | 1.08469E+02 
445| 17840  | 2.87100E+03 | 3.44700E+03 | 3.06931E+03 | 2.87100E+03 | 1.55974E+02 
446| 17880  | 2.87100E+03 | 3.50000E+03 | 3.10544E+03 | 2.87100E+03 | 2.05541E+02 
447| 17920  | 2.87100E+03 | 3.53500E+03 | 3.13543E+03 | 2.87100E+03 | 2.28315E+02 
448| 17960  | 2.87100E+03 | 3.52050E+03 | 3.11606E+03 | 2.87100E+03 | 1.90174E+02 
449| 18000  | 2.87100E+03 | 3.51400E+03 | 3.07991E+03 | 2.87100E+03 | 1.79086E+02 
450| 18040  | 2.87100E+03 | 3.58400E+03 | 3.04197E+03 | 2.87100E+03 | 1.87931E+02 
451|